In [1]:
import os
import pandas as pd
import qdrant_client
from dotenv import load_dotenv
from langchain.schema import Document
from langchain.vectorstores import Qdrant
from langchain.embeddings import OpenAIEmbeddings
from qdrant_client.http.models import VectorParams, Distance
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [2]:
# Load environment variables
load_dotenv()

QDRANT_HOST = os.getenv("QDRANT_HOST")
API_KEY = os.getenv("API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
QDRANT_COLLECTION_NAME = "imdb"

In [3]:
# Load IMDb dataset
df = pd.read_csv("C:/Users/saisu/Documents/Learning/RAG_project_imdb/data/imdb_top_1000.csv")


In [4]:
# Convert rows into LangChain Document format
documents = [
    Document(
        metadata={"title": row["Series_Title"], "year": row["Released_Year"], "genre": row["Genre"], "rating": row["IMDB_Rating"]},
        page_content=(
            f"Movie: {row['Series_Title']}, Released: {row['Released_Year']}, Genre: {row['Genre']}, "
            f"Rating: {row['IMDB_Rating']}, Director: {row['Director']},  Overview: {row['Overview']}"
            f"Starring: {row['Star1']}, {row['Star2']}, {row['Star3']}, {row['Star4']}."
        )
    )
    for _, row in df.iterrows()
]

In [3]:
# Initialize Qdrant Client
client = qdrant_client.QdrantClient(url=QDRANT_HOST, api_key=API_KEY, timeout=120)

# Check if collection exists, then create it
if not client.collection_exists(QDRANT_COLLECTION_NAME):
    client.create_collection(
        collection_name=QDRANT_COLLECTION_NAME,
        vectors_config=VectorParams(size=1536, distance=Distance.COSINE)
    )


In [4]:
# Initialize embeddings
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
embeddings = OpenAIEmbeddings()

# Connect vector store
vector_store = Qdrant(
    client=client,
    collection_name=QDRANT_COLLECTION_NAME,
    embeddings=embeddings
)

C:\Users\saisu\AppData\Local\Temp\ipykernel_11700\2627495524.py:3: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings()
C:\Users\saisu\AppData\Local\Temp\ipykernel_11700\2627495524.py:6: LangChainDeprecationWarning: The class `Qdrant` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-qdrant package and should be used instead. To use it run `pip install -U :class:`~langchain-qdrant` and import as `from :class:`~langchain_qdrant import Qdrant``.
  vector_store = Qdrant(


In [7]:
# Define text splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=50,
    separators=["\n"]
)

# Process documents and split text correctly
chunked_documents = []
for doc in documents:
    chunks = text_splitter.split_text(doc.page_content)  # Use page_content instead of passing Document object
    chunked_documents.extend(chunks)

# Display some chunked samples
print(chunked_documents[:5])

['Movie: The Shawshank Redemption, Released: 1994, Genre: Drama, Rating: 9.3, Director: Frank Darabont,  Overview: Two imprisoned men bond over a number of years, finding solace and eventual redemption through acts of common decency.Starring: Tim Robbins, Morgan Freeman, Bob Gunton, William Sadler.', "Movie: The Godfather, Released: 1972, Genre: Crime, Drama, Rating: 9.2, Director: Francis Ford Coppola,  Overview: An organized crime dynasty's aging patriarch transfers control of his clandestine empire to his reluctant son.Starring: Marlon Brando, Al Pacino, James Caan, Diane Keaton.", 'Movie: The Dark Knight, Released: 2008, Genre: Action, Crime, Drama, Rating: 9.0, Director: Christopher Nolan,  Overview: When the menace known as the Joker wreaks havoc and chaos on the people of Gotham, Batman must accept one of the greatest psychological and physical tests of his ability to fight injustice.Starring: Christian Bale, Heath Ledger, Aaron Eckhart, Michael Caine.', 'Movie: The Godfather: P

In [8]:
vector_store.add_texts(chunked_documents)

['6fdb3e4e71504ce898fa8888d4c89371',
 'd07d0f2bde5440dcafb3b9e717fd265d',
 '56d3b198884f4e85abf0152c88572594',
 'db34f2c0699a4778b3c6e792627fbae4',
 '4b05143de26d4e6d8299789a5d3083ba',
 '79eeafc8b9cd40568fd26b53d334d5b3',
 '14c0273e71fb42428c4a15ffbccea520',
 '6fc779afc5ba40eda8e45d7284dee786',
 '8dd27b03d0f1407e86261670e5d5d8cf',
 'bf8a266c6a164bb6a31d946909f69c0b',
 'c666e83759794b2eb6115dd2afca701d',
 '16db9598cfcf40c9be06a993129e3e65',
 '56264a0fd8114b18ac5be5cd7e44ae33',
 'c7bd4627af0947399cd04c5288961bb4',
 'c50bb8865cce4a0588c3bbae5b0587d4',
 '722ffbd93f9441cd90a44f4bb50227a2',
 '8446b17952144f7894663e1eb9e0f4ad',
 'b6a68a5925534d1b88b901b8083ee183',
 'cead36a0d7ce46c5a72219a55dd47bef',
 '8ae0d1bdc6334d23856b71187be33b8f',
 '34db1d1cda6f491b8aba745d7a7e3ca0',
 '497f7e041341416ca3cbcb39dc9979f7',
 '6c1dc2685a3f43b69736f8db19ee838e',
 '8d32f719a30d4f8fafc466d5b3da673b',
 '3227f50092a044b8ba5d9ecf26959f91',
 '4646725b0f1348939c728e5232cf5e44',
 '865f366cf10043c6a199755d58e04bf8',
 

In [5]:
# plug vector store into retrieval chain

from langchain.chains import RetrievalQA
from langchain_groq import ChatGroq

qa=RetrievalQA.from_chain_type(
    llm=ChatGroq(model='deepseek-r1-distill-llama-70b', temperature=0.5),
    chain_type='stuff',
    retriever=vector_store.as_retriever()
)

In [6]:
import re

def remove_think_tags(text):
    return re.sub(r"<think>.*?</think>", "", text, flags=re.DOTALL).strip()

In [7]:
def get_response(query):
    response=qa.run(query)
    if "<think>" in response:
        response=remove_think_tags(response)
    return response

In [8]:
query="Inception, who are actors in it?"
print(get_response(query=query))

C:\Users\saisu\AppData\Local\Temp\ipykernel_11700\869843029.py:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response=qa.run(query)


The actors in *Inception* include:

- Leonardo DiCaprio  
- Joseph Gordon-Levitt  
- Elliot Page (formerly known as Ellen Page)  
- Ken Watanabe
